# Calculate relevant variable

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely.geometry as sg
import shapely.ops as so
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

import networkx as nx
import momepy

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import upc_sw.poly_utils as poly_utils

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

## Settings

In [ ]:
# Paths
bgt_road_file = '../datasets/bgt/bgt_voetpad.gpkg'
point_cloud_file = '../datasets/output/sidewalks_with_obstacles.gpkg'
segments_file = '../datasets/output/sidewalk_segments.gpkg'
output_file = '../datasets/output/final_output_segments.geojson'
output_legend = '../datasets/output/final_output_legend.csv'
output_image = '../datasets/output/final_output_image.png'

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
CRS = 'epsg:28992'

# Boundary for filtering out (in meters)
min_path_width = 0.4 

# Boundaries between the final colors green/orange/red (in meters)
width_lower = 0.9  
width_upper = 1.5 

# Maximum distance between intended start point and start node (in meters)
max_dist = 3 

# Maximum length of linestring (in meters), otherwise cut
max_ls_length = 90

# Minimum interior size to remain in BGT data
min_interior_size = 10

## Import data

In [ ]:
# Read BGT data
df_bgt = gpd.read_file(bgt_road_file, crs=CRS)

In [ ]:
# Read sidewalk with obstacle data (point cloud)
df_pc = gpd.read_file(point_cloud_file, crs=CRS)

In [ ]:
# Read lines with widths (calculated in previous notebook)
df_segments = gpd.read_file(segments_file, crs=CRS)

## Process data

### Remove too narrow paths

In [ ]:
# Apply minimal path width
df_segments_wide = df_segments[df_segments['min_width'] > min_path_width].reset_index(drop=True)
print(df_segments.shape)
print(df_segments_wide.shape)

In [ ]:
# Remove short lines
mls_per_id = poly_utils.create_mls_per_sidewalk(df_segments_wide, crs=CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(
                        lambda x: poly_utils.remove_short_lines(x))

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_segments_wide = df_segments_wide.merge(long_segments_df, how='inner')
df_segments_wide.shape

### Use color codes

In [ ]:
conditions = [
    (df_segments_wide['min_width'] < width_lower),
    (df_segments_wide['min_width'] >= width_lower) & (df_segments_wide['min_width'] < width_upper),
    (df_segments_wide['min_width'] >= width_upper)
]

values = ['red', 'orange', 'lightgreen']

In [ ]:
df_segments_wide['min_width_color'] = np.select(conditions, values)
df_segments_wide['min_width_color'].value_counts()

In [ ]:
# Add width factor, for calculating the weights of the paths later
values = [1000000, 1000, 1]
df_segments_wide['min_width_factor'] = np.select(conditions, values)

### Take point cloud coverage into account

In [ ]:
# TO DO

### Create centerlines without obstacles

In [ ]:
df_bgt['geometry_no_holes'] = df_bgt.progress_apply(
    lambda row: poly_utils.remove_interiors(row.geometry, min_interior_size), axis=1)
df_bgt = df_bgt.set_geometry('geometry_no_holes')

In [ ]:
# Merge sidewalk polygons (optional, but should be in line with notebook 5 'merge_sidewalks' parameter)
df_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry_no_holes'].unary_union))
df_bgt = gpd.GeoDataFrame(df_bgt.geometry.explode()) 

In [ ]:
# Calculate centerlines
df_bgt['centerlines'] = df_bgt.progress_apply(
    lambda row: Centerline(row.geometry, interpolation_distance=0.5), axis=1)
df_bgt = df_bgt.set_geometry('centerlines')

#### Remove dead-ends

In [ ]:
df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(so.linemerge)

In [ ]:
df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(poly_utils.remove_short_lines)

## Create relevant variable

In [ ]:
# Create final dataframe
final_df = pd.DataFrame()

for i in range(len(df_bgt['centerlines'])):
    
    # Get centerline
    my_centerline = df_bgt['centerlines'].values[i]
    
    # Create dataframe with linestrings of centerline
    centerline_df = poly_utils.create_df_centerlines(my_centerline)
    
    # Cut linestrings that are too long
    centerline_df = poly_utils.shorten_linestrings(centerline_df, max_ls_length)
        
    # Get sidewalk polygon for this centerline
    my_sidewalk = df_bgt['geometry'].values[i]
    
    # Create graph for all paths withing this sidewalk polygon
    df_sidewalk = df_segments_wide[df_segments_wide['geometry'].within(my_sidewalk)].reset_index(drop=True)
    G = momepy.gdf_to_nx(df_sidewalk, approach="primal", multigraph=True)
    
    for j in range(len(centerline_df['geometry'])):
        
        # Get line
        my_line = centerline_df.iloc[[j]]['geometry'].values[0]
        
        # Get origin and destination location
        if len(list(G.nodes)) > 0:
            origin_point, dest_point = my_line.boundary
            origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G.nodes)))[1]
            dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G.nodes)))[1]

            # Get origin and destination node
            if (origin_point.distance(origin_node_loc) > max_dist) or (dest_point.distance(dest_node_loc) > max_dist):
                print('origin and/or destination node too far from line start/end for line (j)', j, 'in sidewalk (i)', i) 
            else:    
                origin_node = (origin_node_loc.x, origin_node_loc.y)
                dest_node = (dest_node_loc.x, dest_node_loc.y)   

                # Get weight of optimal route in graph
                try:
                    route_weight = nx.shortest_path_length(G, origin_node, dest_node, 
                                                           weight='min_width_factor')
                    centerline_df['route_weight'][j] = route_weight
                except nx.NetworkXNoPath:
                    print('no route found for line (j)', j, 'in sidewalk (i)', i)
                    centerline_df['route_weight'][j] = 1000000000
        else:
            print('network has zero nodes')

    # Append data to final dataframe
    final_df = final_df.append(centerline_df)
final_df = final_df.reset_index()    

In [ ]:
# Get final color of the routes
final_df['final_color'] = final_df.progress_apply(
    lambda row: poly_utils.get_route_color(row.route_weight), axis=1)

In [ ]:
final_df['final_color'].value_counts()

In [ ]:
final_df_group = final_df.groupby('final_color').sum()
final_df_group['length_perc'] = round(100 * final_df_group['length'] / sum(final_df_group['length']), 1)
final_df_group[['length_perc']]

## Post-process output

In [ ]:
# Get dataframe with only valid lines
final_df_select = final_df[final_df['final_color'].isin(['lightgreen', 'orange', 'red', 'purple', 'black'])]

In [ ]:
# Recolor paths with unknown widths
final_df_select['final_color'] = final_df_select['final_color'].replace(['black', 'purple'], 'grey')

In [ ]:
# Select only relevant columns
final_df_select = final_df_select[['geometry', 'final_color']]

In [ ]:
# Add object ID
final_df_select['object_id'] = final_df_select.index

In [ ]:
# Adjust crs: maps.amsterdam.nl requested WGS84 (Lng-Lat) 
final_df_select = final_df_select.set_crs(CRS) 
df_projected = final_df_select.to_crs('epsg:4326') 

## Store output

In [ ]:
with open(output_file, 'w') as f:
    f.write(df_projected.to_json())

## Create and store legend table

In [ ]:
df_legend = pd.DataFrame(np.array([['lightgreen', 'toegankelijk (>' + str(width_upper) + 'm)', 'accessible (>' + str(width_upper) + 'm)'], 
                                   ['orange', 'smal (' + str(width_lower) + '-' + str(width_upper) + 'm)', 'narrow (' + str(width_lower) + '-' + str(width_upper) + 'm)'],
                                   ['red', 'niet toegankelijk (<' + str(width_lower) + 'm)', 'not accessible (<' + str(width_lower) + 'm)'],
                                   ['grey', 'onbekend', 'unknown']
                                  ]), columns = ['final_color', 'label_NL', 'label_ENG'])

In [ ]:
df_legend.to_csv(output_legend, index=False)

## Visualisation

In [ ]:
# Boundaries for plotting a subset of the data
#x_min = 125730 # area 4 
#x_max = 125810
#y_min = 489820
#y_max = 489960
#x_min = 122700 # area 13
#x_max = 122800
#y_min = 490240
#y_max = 490340
#x_min = 114680 # area 8
#x_max = 114830
#y_min = 487520
#y_max = 487690
x_min = 122400 # larger area east
x_max = 123600
y_min = 485400
y_max = 486000

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5), dpi=600, 
                                    frameon=False, constrained_layout=True)

# Plot process step 1
df_bgt.set_geometry('geometry').plot(ax=ax1, color="silver")
df_pc.plot(ax=ax1)
df_segments_wide.plot(ax=ax1, linewidth=0.5, color=df_segments_wide['min_width_color'])
ax1.axis('off')

# Plot process step 2
df_bgt.set_geometry('geometry').plot(ax=ax2, color="silver")
df_pc.plot(ax=ax2)
final_df.plot(ax=ax2, linewidth=1, color=final_df.final_color)  
ax2.axis('off')

# Plot process step 3
df_bgt.set_geometry('geometry').plot(ax=ax3)
final_df_select.plot(ax=ax3, linewidth=1, color=final_df_select.final_color)  
ax3.axis('off')

# Create legends
acc = mpatches.Patch(color='lightgreen', label='accessible (>' + str(width_upper) + 'm)')
narrow = mpatches.Patch(color='orange', label='narrow (' + str(width_lower) + '-' + str(width_upper) + 'm)')
notacc = mpatches.Patch(color='red', label='not accessible (<' + str(width_lower) + 'm)')
rno = mpatches.Patch(color='darkred', label='no route possible')
rnan = mpatches.Patch(color='purple', label='node too far')
rl0 = mpatches.Patch(color='black', label='route length 0')
runk = mpatches.Patch(color='grey', label='unknown')
legend1 = plt.legend(handles=[acc,narrow,notacc,rno,rnan,rl0], 
           bbox_to_anchor=(-0.5, -0.5, 0.5, 0.5))
plt.legend(handles=[acc,narrow,notacc,runk], 
           bbox_to_anchor=(0.5, -0.5, 0.5, 0.5))
plt.gca().add_artist(legend1)

# Set plot limits
ax1.set_xlim([x_min, x_max])   
ax1.set_ylim([y_min, y_max])
ax2.set_xlim([x_min, x_max])
ax2.set_ylim([y_min, y_max])
ax3.set_xlim([x_min, x_max])
ax3.set_ylim([y_min, y_max])

plt.savefig(output_image, bbox_inches='tight')
plt.show()

In [ ]:
#x_min = 122400 # subset A
#x_max = 122700
#y_min = 485700
#y_max = 486000
#x_min = 122700 # subset B
#x_max = 123000
#y_min = 485700
#y_max = 486000
#x_min = 123000 # subset C
#x_max = 123300
#y_min = 485700
#y_max = 486000
#x_min = 123300 # subset D
#x_max = 123600
#y_min = 485700
#y_max = 486000
#x_min = 122400 # subset E
#x_max = 122700
#y_min = 485400
#y_max = 485700
#x_min = 122700 # subset F
#x_max = 123000
#y_min = 485400
#y_max = 485700
#x_min = 123000 # subset G
#x_max = 123300
#y_min = 485400
#y_max = 485700
x_min = 123300 # subset H
x_max = 123600
y_min = 485400
y_max = 485700

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5), dpi=600, 
                                    frameon=False, constrained_layout=True)

# Plot process step 1
df_bgt.set_geometry('geometry').plot(ax=ax1, color="silver")
df_pc.plot(ax=ax1)
df_segments_wide.plot(ax=ax1, linewidth=0.5, color=df_segments_wide['min_width_color'])
ax1.axis('off')

# Plot process step 2
df_bgt.set_geometry('geometry').plot(ax=ax2, color="silver")
df_pc.plot(ax=ax2)
final_df.plot(ax=ax2, linewidth=1, color=final_df.final_color)  
ax2.axis('off')

# Plot process step 3
df_bgt.set_geometry('geometry').plot(ax=ax3)
final_df_select.plot(ax=ax3, linewidth=1, color=final_df_select.final_color)  
ax3.axis('off')

# Create legends
acc = mpatches.Patch(color='lightgreen', label='accessible (>' + str(width_upper) + 'm)')
narrow = mpatches.Patch(color='orange', label='narrow (' + str(width_lower) + '-' + str(width_upper) + 'm)')
notacc = mpatches.Patch(color='red', label='not accessible (<' + str(width_lower) + 'm)')
rno = mpatches.Patch(color='darkred', label='no route possible')
rnan = mpatches.Patch(color='purple', label='node too far')
rl0 = mpatches.Patch(color='black', label='route length 0')
runk = mpatches.Patch(color='grey', label='unknown')
legend1 = plt.legend(handles=[acc,narrow,notacc,rno,rnan,rl0], 
           bbox_to_anchor=(-0.5, -0.5, 0.5, 0.5))
plt.legend(handles=[acc,narrow,notacc,runk], 
           bbox_to_anchor=(0.5, -0.5, 0.5, 0.5))
plt.gca().add_artist(legend1)

# Set plot limits
ax1.set_xlim([x_min, x_max])   
ax1.set_ylim([y_min, y_max])
ax2.set_xlim([x_min, x_max])
ax2.set_ylim([y_min, y_max])
ax3.set_xlim([x_min, x_max])
ax3.set_ylim([y_min, y_max])

output_image_sub = '../datasets/output/final_output_image_H.png'
plt.savefig(output_image_sub, bbox_inches='tight')

plt.show()